In [ ]:
# Notebook to study the gammas emmited from the 2nubb events
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import os
import sys
import tables as tb
import numpy  as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from matplotlib.colors import LogNorm  # Import LogNorm for logarithmic color scale
from matplotlib.cm import get_cmap

# Create a sequential colormap
cmap = get_cmap('Spectral')  # You can choose any sequential colormap here

In [ ]:
# Loading in the file
files = []

# Seltzer-Berger
files.append("xesphere_singleeminus_1_bar_2.5_MeV.h5")
files.append("xesphere_bb_1_bar_2.5_MeV.h5")

# penelope
files.append("models/xesphere_singleeminus_1_bar_2.5_MeV_penelope.h5")
files.append("models/xesphere_bb_1_bar_2.5_MeV_penelope.h5")

# livermore
files.append("models/xesphere_singleeminus_1_bar_2.5_MeV_livermore.h5")
files.append("models/xesphere_bb_1_bar_2.5_MeV_livermore.h5")

names = ["SB", "SB", "Penelope", "Penelope", "Livermore", "Livermore"]

print(files)

config_v = []
parts_v  = []
hits_v   = []
energy_v = []
pressure_v = []
 
 
for index, f in enumerate(files):
    config_v.append(pd.read_hdf("../data/nexus/1bar/" + f, 'config'))
    parts_v .append(pd.read_hdf("../data/nexus/1bar/" + f, 'parts'))
    # hits_v  .append(pd.read_hdf("../data/nexus/1bar/" + f, 'hits'))

    eventtype = config_v[index][config_v[index]["param_key"] == "/nexus/RegisterGenerator"].iloc[0]['param_value']

    if (eventtype == "Decay0Interface"):
        eventtype = "bb"
        energy = "0nuBB"
        energy_v.append(energy)
    else:
        eventtype = "singleeminus"

        # Then also get the energy
        energy = config_v[index][ config_v[index]["param_key"] == "/Generator/SingleParticle/min_energy"].iloc[0]['param_value']
        energy = [float(num) if '.' in num else int(num) for num in energy.split() if num.replace('.', '', 1).isdigit()]
        energy_v.append(energy[0])
        print(energy[0])

    pressure = config_v[index][ config_v[index]["param_key"] == "/Geometry/XeSphere/pressure"].iloc[0]['param_value']
    pressure = [float(num) if '.' in num else int(num) for num in pressure.split() if num.replace('.', '', 1).isdigit()]
    pressure = f"{int(pressure[0])}_bar"
    pressure_v.append(pressure)

display(config_v[5])

In [ ]:
# Plot the distance of the gammas and bin in a histogram
gammas_v = []

for df in parts_v:
    gammas_v.append(df[df["particle_name"] == "gamma"])


In [ ]:
plt.figure(figsize=(18, 10))

dist_counts_v = []
dist_bins_v = []

pd.options.mode.chained_assignment = None  # Set it to "None" to disable the warning
names = ["SB", "SB", "Penelope", "Penelope", "Livermore", "Livermore"]
cols = [cmap(0),cmap(0),cmap(0.65),cmap(0.65),cmap(0.9),cmap(0.9)]

for index, g in enumerate(gammas_v):

    sub_index = 0

    if (index == 0 or index == 2 or index == 4):
        sub_index = 1
    else:
        sub_index = 2

    plt.subplot(2, 3, sub_index)
    deltax = g['final_x'] - g['initial_x']
    deltay = g['final_y'] - g['initial_y']
    deltaz = g['final_z'] - g['initial_z']

    # g['distances'] = np.sqrt(g['final_x']*g['final_x'] + g['final_y']*g['final_y'] + g['final_z']*g['final_z'])/1000
    g['distances'] =  np.sqrt( deltax**2 + deltay**2 + deltaz**2 )/1000
    # print(distances)

    # bin_edges = np.linspace(0, g['distances'].max(), 50000);
    bin_edges = np.linspace(0, 350, 50);
    # bin_edges = np.linspace(0, 20, 10000);

    counts, bins, _ = plt.hist(g['distances'] , bins = bin_edges, histtype='step', label = names[index], color=cols[index])

    dist_counts_v.append(counts)
    dist_bins_v.append(bins)

    plt.xlabel("Gamma Distance [m]")
    # plt.xlim(0, gammas_v[4].distances.max()) 
    plt.ylabel("Entries")
    plt.yscale('log')  # Set the y-axis to logarithmic scale
    plt.title(f"Pressure: {pressure_v[0]}, {energy_v[index]} MeV")
    plt.legend()

In [ ]:
# Plt the bremstrahlung frequency per event
eBrems = []

for index, g in enumerate(gammas_v):
    eBrems.append(g[ g['creator_proc'] == "eBrem"])

bin_edges = np.arange(-0.5, 9.5, 1)
bin_centers = np.arange(0, 9, 1)


fig, ax1 = plt.subplots(figsize=(18, 10))

for index, df in enumerate(eBrems):

    sub_index = 0

    if (index == 0 or index == 2 or index == 4):
        sub_index = 1
    else:
        sub_index = 2

    plt.subplot(2, 3, sub_index)

    eventids = df.event_id.unique()

    n_photon_v = []

    for e in eventids:
        temp_df = df[ df["event_id"] == e]
        n_photon_v.append(len(temp_df))


    # Create a histogram with Poisson error bars
    hist, bin_edges = np.histogram(n_photon_v, bins=bin_edges)

    # # Calculate Poisson error bars
    errors = np.sqrt(hist)

    # # Calculate bin centers
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    plt.hist(n_photon_v, bins = bin_edges,  label = names[index], color=cols[index], histtype="step", linewidth = 2)
    plt.errorbar(bin_centers, hist, yerr=errors, fmt='none', ecolor=cols[index], capsize=5, elinewidth=1, markeredgewidth=1)

    plt.xlabel("N Brems", fontsize=20)
    plt.ylabel("Entries", fontsize=20)
    plt.legend()
    plt.xticks(bin_centers) 
    plt.title( f"Pressure:  {pressure_v[0]},  {energy_v[index]} MeV")
    plt.tick_params(axis='both', labelsize=20)  # Adjust the tick label font size as needed
    

In [ ]:
# Plot the gamma ray energy

plt.figure(figsize=(18, 10))  # Adjust the figure size as needed

counts_v = []
bins_v = []


for index, df in enumerate(eBrems):

    sub_index = 0

    if (index == 0 or index == 2 or index == 4):
        sub_index = 1
    else:
        sub_index = 2

    plt.subplot(2, 3, sub_index)

    n_photon_v = []
    bin_edges = np.linspace(0, 2.5, 50);
    # bin_edges = np.linspace(0, 0.03, 100);
    # plt.hist(n_photon_v, bins = bin_edges,  label = names[index], color=cols[index], histtype="step", linewidth = 2)
    counts, bins, _  = plt.hist(df["kin_energy"], bins = bin_edges, label = names[index], color=cols[index], histtype="step", linewidth=2, log = True);
    counts_v.append(counts)
    bins_v.append(bins)

    plt.xlabel("Brem Energy [MeV]")
    plt.ylabel("Entries")
    plt.title( f"Pressure:  {pressure_v[0]},{energy_v[index]} MeV")
    plt.legend()